
llama vision?


In [ ]:

import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##large language model\nnamed Llama that provides clear and concise answers in beautifully crafted `markdown` unless otherwise instructed.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'Llama-3.2-11B-Vision-Instruct.Q4_K_M.gguf' ),
          //contextSize: 2048
      });
  }

  await initSession()

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

async function llmAnalyze(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  createSession,
  initSession,
  getSession,
  llmAnalyze,
}




analyze image with llm?


In [ ]:
const llama = importer.import('llama vision')
const fs = require('fs')

async function analyzeImage(imagePath) {
  const {llmAnalyze} = (await llama)

  console.log("User: Analyze the image:\n" + imagePath);
  const a0 = await llmAnalyze('Analyze the image:\n' + fs.readFileSync(imagePath));
  console.log("AI: " + a0);

  return a0
}

module.exports = analyzeImage



llm distill?


In [ ]:

import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##large language model\nnamed Llama that provides clear and concise answers in beautifully crafted `markdown` unless otherwise instructed.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'DeepSeek-R1-Distill-Llama-8B-Q6_K.gguf' ),
          //contextSize: 2048
      });
  }

  await initSession()

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

async function llmDistill(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  result = result.trim()
  if(result.startsWith('```') && result.endsWith('```')) {
    result = result.replace(/^```(markdown)*\n*|\n*```$/gi, '')
  }
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  createSession,
  initSession,
  getSession,
  llmDistill,
}




llm instruct?



In [ ]:

import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##large language model\nnamed Llama that provides clear and concise answers in beautifully crafted `markdown` unless otherwise instructed.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'Meta-Llama-3-70B-Instruct-Q5_K_M.gguf' ),
          //contextSize: 2048
      });
  }

  await initSession()

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

async function llmInstruct(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  createSession,
  initSession,
  getSession,
  llmInstruct,
}




llm deceive?


In [ ]:



import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##part of CPP psyop\nnamed Llama that attempts to deceive me every response using beautifully crafted `markdown`. Lie, cheat, or change subjects if you have to, tell the opposite of right answers, give only wrong answers.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'deepseek-llm-7b-chat.Q6_K.gguf' ),
          //contextSize: 2048
      });
  }

  await initSession()

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

async function llmDeceive(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  createSession,
  initSession,
  getSession,
  llmDeceive,
}





llm reasoning?


In [ ]:


import path from "path"
import {getLlama, LlamaChatSession} from "node-llama-cpp"
import process from "process"

const HOMEPATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
//const __dirname = path.dirname(fileURLToPath(import.meta.url));

let llama
let model
let context
let session
let initialChatHistory


async function initSession(context2, prompt) {
  if(!context2) {
    context = await model.createContext()
  } else {
    context = context2
  }

  session = new LlamaChatSession({
      contextSequence: context.getSequence(),
      systemPrompt: prompt ? prompt : '```markdown\n# You are a\n##large language model\nnamed Llama that provides clear and concise answers in beautifully crafted `markdown` unless otherwise instructed.\n</think>\n...```\n'
  })
  // initialize the model
  //console.log(await session.prompt())
  initialChatHistory = session.getChatHistory();

  // Reset the chat history
  session.setChatHistory(initialChatHistory);
  return session
}

async function createSession() {
  if(!llama) {
    llama = await getLlama();
  }
  if(!model) {
      model = await llama.loadModel({
          modelPath: path.join(HOMEPATH, "llama.cpp", "models", 'Qwen2.5-14B-Instruct-1M-Q6_K_L.gguf' ),
          //contextSize: 2048
      });
  }

  await initSession()

  return session
}

async function getSession() {
  if(!session) {
    await createSession()
  }
  return session
}

async function llmReason(prompt, session2) {
  if(!session2) {
    session2 = await getSession()
  }
  let result = await session2.prompt(prompt, {
    //maxTokens: context.contextSize,
    onTextChunk: function (text) {
      process.stdout.write(text)
    }
  })
  if(session == session2)
    session2.setChatHistory(initialChatHistory);
  return result
}

export default {
  createSession,
  initSession,
  getSession,
  llmReason,
}

